In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

hf_state = [1,1,1,1,0,0,0,0]
qubits = 8

dev = qml.device('default.qubit', wires=8)

@qml.qnode(dev)
def circuit(weight):
    qml.BasisState(hf_state, wires=range(qubits))
    qml.FermionicSingleExcitation(weight, wires=[2,6])
    #qml.DoubleExcitation(weight, wires=[0,1,2,3])
    #qml.SingleExcitation(weight, wires=[2,0])
    
    return qml.state()

weight = 10.0
print(circuit(weight))
print(qml.draw(circuit, level="device", max_length=1000)(weight))


In [ ]:
-7.77775928 - (-7.777758247643)

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

dev = qml.device('default.qubit', wires=8)
#hf_state = [1,1,1,1,0,0,0,0]
hf_state = [0,0,0,0,1,1,1,1]

@qml.qnode(dev)
def circuit(weight):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    #qml.FermionicSingleExcitation(weight, wires=[0,2])
    qml.SingleExcitation(weight, wires=[0,4])
    #qml.FermionicDoubleExcitation(weight, wires1=[0,1], wires2=[2,3])
    #qml.DoubleExcitation(weight, wires=[0,1,2,3])
    #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.state()

weight = 0.9
print(circuit(weight))
print(qml.draw(circuit, level="device", max_length=1000)(weight))


In [ ]:
#-0.43496553  0.9004471 - 4th line

In [ ]:
import numpy as np
import pennylane as qml
from scipy.optimize import minimize

import pennylane as qml

from pennylane import qchem
from pennylane import numpy as np




X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 3.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "openfermion")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
qubits = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")



dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

# Define the Ash excitation structure (as per your example)
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [2,6]]
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [3, 7], [0,4]]
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5]]

ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]
params = np.zeros(len(ash_excitation), requires_grad=True)

dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            qml.FermionicDoubleExcitation(weight = params[i], wires1=excitations[2:][::-1], wires2=excitations[:2][::-1])
        else:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = ash_excitation[i])
    return qml.expval(H)


#dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
#@qml.qnode(dev)
def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    
    # Compute the gradient of the energy with respect to the parameters
    #grad = qml.grad(circuit)(params, ash_excitation, hf_state, qubits, H)
    
        # Calculate gradient norm (L2 norm)
    #grad_norm = np.linalg.norm(grad)
    
    # Calculate maximum gradient
    #grad_max = np.max(np.abs(grad))
    
    # Debugging: Print the gradient information
    print(f"Energy: {energy}")
    #print(f"Gradient: {grad}")
    #print(f"Gradient Norm: {grad_norm}")
    #print(f"Maximum Gradient: {grad_max}")
    return energy



params_initial = np.zeros(len(ash_excitation))
#bounds = ([(0, 0.97)] * len(ash_excitation))

# Function to minimize
def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

# Using scipy.optimize.minimize with L-BFGS-B
result = minimize(
    fun=minimize_function,   # The cost function
    x0=params_initial,       # Initial guess
    jac=True,                 # Enable gradient computation
    method='L-BFGS-B',        # Use L-BFGS-B optimizer
    #bounds=bounds,       # Use L-BFGS-B optimizer
    options={'disp': True, 'maxiter': 100, 'gtol' : 1e-12}  # Display options and max iterations
)

# Extract the result
optimized_params = result.x
minimized_energy = result.fun
Gradient = result.jac   

print(f"Optimized parameters: {optimized_params}")
print(f"Minimized energy: {minimized_energy}")


## Adaptive optimizer

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem



symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

n_electrons = 4
singles, doubles = qml.qchem.excitations(n_electrons, qubits)
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations
print(len(operator_pool))

hf_state = qml.qchem.hf_state(n_electrons, qubits)
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

print(circuit())

opt = qml.AdaptiveOptimizer(param_steps = 6)
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool, drain_pool=False)
    print('Energy:', energy)
    print(qml.draw(circuit, show_matrices=False)())
    print('Largest Gradient:', gradient)
    print()
    if gradient < 1e-4:
        break

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886  #5.0 Angstrom in Bohr 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
# Calculation of the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
# Calculation of the excitation operators
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)  #HF state : [1,1,1,1,0,0,0,0] - [alpha, beta, alpha, beta, alpha, beta, alpha, beta]
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
print('HF state is', circuit(hf_state, electrons, qubits, H))

print('The original hamiltonian is', H)
#Putting all the functions here
#1.1091571486954503

## Generation of Hamiltonian from Openfermion adn conversion to Pennylane

In [ ]:
import pennylane as qml
import openfermion
from pennylane import numpy as np
from openfermionpyscf import run_pyscf

#------------------------
symbols = ["H", "H", "H", "H"]
x = np.array([0.0000000000,  0.0000000000, 5.0,
              0.0000000000,  0.0000000000, 10.0,
              0.0000000000,  0.0000000000, 15.0,
              0.0000000000,  0.0000000000, 20.0])
mult = 1
charge = 0
basis = 'sto-6g'
#core = [0, 1, 2, 3]
active = [0,1,2,3,4, 5]
filename = './H4'
#------------------------

geometry = [[s, tuple(x[3 * i : 3 * i + 3])] for i, s in enumerate(symbols)]
#print(geometry)

molecule = openfermion.MolecularData(geometry, basis, mult, charge, filename=filename)


run_pyscf(molecule, run_scf=1, verbose=0)

molecule_ = openfermion.MolecularData(filename=filename)

ht = molecule_.get_molecular_hamiltonian()

hf = openfermion.transforms.get_fermion_operator(ht) # fermion hamiltonian

hq = openfermion.transforms.jordan_wigner(hf) # qubit hamiltonian

#hp = qml.qchem.convert_observable(hq) # PennyLane hamiltonian
hp = qml.from_openfermion(hq)

print(hp)

## H4-1A

In [4]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
#symbols = ["Li", "H"]
r_bohr = 1.0*1.8897259886  #5.0 Angstrom in Bohr 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
#coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
# Calculation of the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")


electrons = 4
orbitals = 8
# Calculation of the excitation operators
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)  #HF state : [1,1,1,1,0,0,0,0] - [alpha, beta, alpha, beta, alpha, beta, alpha, beta]
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
print('HF state is', circuit(hf_state, electrons, qubits, H))

print('The original hamiltonian is', H)
#Putting all the functions here
#1.1091571486954503

Singles are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]
Doubles are [[0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
Total number of excitations = 26
Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -2.1124607252577317
The original hamiltonian is -0.3528372123109008 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.18311695676742126 * Z(0) + 0.005593161752353127 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.005593161752353127 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.08955343029607518 * Z(2) + 0.06964291217842293 * (Z(0) @ Z(2)) + 0.0173376567184301 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.0173376567184301 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.01032965410286123 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.01032965410286123 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.076381

In [3]:
# Commutator calculation for HF state
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    #res = (qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation for other states except HF state
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w) #Calculating the commutator
    return qml.expval(res)

# Energy calculation 
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def ash(params, ash_excitation, hf_state, qubits, H):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])  #Applying the double excitation and their parameters
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i]) #Applying the single excitation and their parameters
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))

    return qml.expval(H)  #Calculating the expectation value of the Hamiltonian

# Calculation of New state, same as the above function but with the state return

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def new_state(hf_state, ash_excitation, fparams, params):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])

        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))

            

    return qml.state()

In [5]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)  #Optimizer

print('The Hamiltonian is ', H)
adit = 18
fparams = []
excitations= []
operator_pool = (singles) + (doubles)  #Operator pool - Singles and Doubles
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)  #Adapt iteration
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)   #Current excitation operator - fermionic one
        w = qml.fermi.jordan_wigner(i)  #JW transformation
        if np.array_equal(k, hf_state): # If the current state is the HF state
            print('Print, if this is activated - HF state')
            current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
        else:
            current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="adjoint") #Cost function definition, gradient calculated automatically and method is backprop
    print('Going to do energy calculation')
    for n in range(300):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation - With the updated parameters
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


The Hamiltonian is  -0.3528372123109008 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.18311695676742126 * Z(0) + 0.005593161752353127 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.005593161752353127 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.08955343029607518 * Z(2) + 0.06964291217842293 * (Z(0) @ Z(2)) + 0.0173376567184301 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.0173376567184301 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.01032965410286123 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.01032965410286123 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.0763817659244268 * Z(4) + 0.08457004538976735 * (Z(0) @ Z(4)) + -0.3301799781494755 * Z(6) + 0.10628194743355107 * (Z(0) @ Z(6)) + 0.1831169567674212 * Z(1) + 0.12416940624433223 * (Z(0) @ Z(1)) + 0.02040883856268705 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.02040883856268705 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.03941337226027998 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.03941337226027998 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.03941337226027998 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 

/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 0.0
The current excitation operator is a⁺(0) a⁺(1) a(6) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.19407825946744464
The current excitation operator is a⁺(0) a⁺(2) a(4) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.0912454843358084
The current excitation operator is a⁺(0) a⁺(3) a(4) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 0.0
The current excitation operator is a⁺(0) a⁺(3) a(4) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.19785581016523934
The current excitation operator is a⁺(0) a⁺(3) a(5) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.10661032582943089
The current excitation operator is a⁺(0) a⁺(3) a(6) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) 